## THR->ALA solvent @ lambda = 0 with flattened torsions and exceptions

In [1]:
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.alchemy import AbsoluteAlchemicalFactory, AlchemicalRegion, AlchemicalState
from openmmtools import states
from simtk import openmm, unit
from openmmtools.mcmc import LangevinSplittingDynamicsMove, GHMCMove
from openmmtools.multistate import ReplicaExchangeSampler, MultiStateReporter
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import numpy as np
import pickle

INFO:rdkit:Enabling RDKit 2020.09.1 jupyter extensions


In [33]:
# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [34]:
# Generate htf for capped THR->ALA in solvent
pdb = app.PDBFile("../../input/thr_vacuum.pdb")

forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
system_generator = SystemGenerator(forcefields=forcefield_files,
                               barostat=barostat,
                               forcefield_kwargs={'removeCMMotion': False,
                                                    'ewaldErrorTolerance': 1e-4,
                                                    'constraints' : app.HBonds,
                                                    'hydrogenMass' : 4 * unit.amus},
                                periodic_forcefield_kwargs={'nonbondedMethod': app.PME},
                                small_molecule_forcefield='gaff-2.11',
                                nonperiodic_forcefield_kwargs=None, 
                                   molecules=None, 
                                   cache=None)
modeller = app.Modeller(pdb.topology, pdb.positions)
modeller.addSolvent(system_generator.forcefield, model='tip3p', padding=9*unit.angstroms, ionicStrength=0.15*unit.molar)
solvated_topology = modeller.getTopology()
solvated_positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
system = system_generator.create_system(solvated_topology)

htf = generate_dipeptide_top_pos_sys(solvated_topology, 
                                         new_res='ALA', 
                                         system=system, 
                                         positions=positions,
                                         system_generator=system_generator, 
                                         conduct_htf_prop=True,
                                         repartitioned=True,
                                         endstate=0,
                                         flatten_torsions=True,
                                         flatten_exceptions=True,
                                         validate_endstate_energy=False)

In [27]:
import pickle
import os
out_dir = '/data/chodera/zhangi/perses_benchmark/neq/10/13/'
pickle.dump(htf, open(os.path.join(out_dir, "13_solvent_0.pickle"), "wb" ))

# Render atom map
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping
atom_map_filename = os.path.join(out_dir, 'solvent_0_atom_map.png')
render_protein_residue_atom_mapping(htf._topology_proposal, atom_map_filename)


In [28]:
import mdtraj as md
import numpy as np
htfs_t = [htf]

top_old = md.Topology.from_openmm(htfs_t[0]._topology_proposal.old_topology)
top_new = md.Topology.from_openmm(htfs_t[0]._topology_proposal.new_topology)
traj = md.Trajectory(np.array(htfs_t[0].old_positions(htfs_t[0].hybrid_positions)), top_old)
traj.save("13_solvent_0_old.pdb")
traj = md.Trajectory(np.array(htfs_t[0].new_positions(htfs_t[0].hybrid_positions)), top_new)
traj.save("13_solvent_0_new.pdb")


## THR->ALA solvent @ lambda = 1 with flattened torsions and exceptions

In [29]:
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.alchemy import AbsoluteAlchemicalFactory, AlchemicalRegion, AlchemicalState
from openmmtools import states
from simtk import openmm, unit
from openmmtools.mcmc import LangevinSplittingDynamicsMove, GHMCMove
from openmmtools.multistate import ReplicaExchangeSampler, MultiStateReporter
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import numpy as np
import pickle

In [30]:
# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [31]:
# Generate htf for capped THR->ALA in solvent
pdb = app.PDBFile("../../input/thr_vacuum.pdb")

forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
system_generator = SystemGenerator(forcefields=forcefield_files,
                               barostat=barostat,
                               forcefield_kwargs={'removeCMMotion': False,
                                                    'ewaldErrorTolerance': 1e-4,
                                                    'constraints' : app.HBonds,
                                                    'hydrogenMass' : 4 * unit.amus},
                                periodic_forcefield_kwargs={'nonbondedMethod': app.PME},
                                small_molecule_forcefield='gaff-2.11',
                                nonperiodic_forcefield_kwargs=None, 
                                   molecules=None, 
                                   cache=None)
modeller = app.Modeller(pdb.topology, pdb.positions)
modeller.addSolvent(system_generator.forcefield, model='tip3p', padding=9*unit.angstroms, ionicStrength=0.15*unit.molar)
solvated_topology = modeller.getTopology()
solvated_positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
system = system_generator.create_system(solvated_topology)

htf = generate_dipeptide_top_pos_sys(solvated_topology, 
                                         new_res='ALA', 
                                         system=system, 
                                         positions=positions,
                                         system_generator=system_generator, 
                                         conduct_htf_prop=True,
                                         repartitioned=True,
                                         endstate=1,
                                         flatten_torsions=True,
                                         flatten_exceptions=True,
                                         validate_endstate_energy=False)

INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{7: 9}, {8: 9}, {9: 9}, {7: 10}, {8: 10}, {9: 10}, {7: 11}, {8: 11}, {9: 11}, {7: 12}, {8: 12}, {9: 12}, {7: 13}, {8: 13}, {9: 13}]
INFO:proposal_generator:{7: 9}
INFO:proposal_generator:{8: 9}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{7: 10}
INFO:proposal_generator:{8: 10}
INFO:proposal_generator:{9: 10}
INFO:proposal_generator:{7: 11}
INFO:proposal_generator:{8: 11}
INFO:proposal_generator:{9: 11}
INFO:proposal_generator:{7: 12}
INFO:proposal_generator:{8: 12}
INFO:proposal_generator:{9: 12}
INFO:proposal_generator:{7: 13}
INFO:proposal_generator:{8: 13}
INFO:proposal_generator:{9: 13}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
INFO:proposal_generator:Finding best map using matching_criterion name
INFO:proposal_genera

making topology proposal


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 4
INFO:geometry:Atom index proposal order is [10, 15, 13, 14]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


generating geometry engine
making geometry proposal from THR to ALA


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 9 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 36 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 42 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1619 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (11, 6, 7, 10, [1, Quantity(value=0.97996

...

INFO:geometry:total reduced potential before atom placement: 9.117927746118994


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -72.55558356848493
INFO:geometry:final reduced energy -63.43765693685564
INFO:geometry:sum of energies: -63.43765582236594
INFO:geometry:magnitude of difference in the energies: 1.1144896916448488e-06
INFO:geometry:Final logp_proposal: -28972.909778678608


added energy components: [('CustomBondForce', 0.5174288400267416), ('CustomAngleForce', 6.782407624225441), ('CustomTorsionForce', 17.086151956175012), ('CustomBondForce', -96.94157198891212)]


INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.5648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.5648, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.5648), unit=nanometer)]
INFO:relative:Determined atom classes.
INFO:relative:Generating old system exceptions dict...
INFO:relative:Generating new system exceptions dict...
INFO:relative:Handling constraint

In [32]:
import pickle
import os
out_dir = '/data/chodera/zhangi/perses_benchmark/neq/10/13/'
pickle.dump(htf, open(os.path.join(out_dir, "13_solvent_1.pickle"), "wb" ))

# Render atom map
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping
atom_map_filename = os.path.join(out_dir, 'solvent_1_atom_map.png')
render_protein_residue_atom_mapping(htf._topology_proposal, atom_map_filename)


In [33]:
import mdtraj as md
import numpy as np
htfs_t = [htf]

top_old = md.Topology.from_openmm(htfs_t[0]._topology_proposal.old_topology)
top_new = md.Topology.from_openmm(htfs_t[0]._topology_proposal.new_topology)
traj = md.Trajectory(np.array(htfs_t[0].old_positions(htfs_t[0].hybrid_positions)), top_old)
traj.save("13_solvent_1_old.pdb")
traj = md.Trajectory(np.array(htfs_t[0].new_positions(htfs_t[0].hybrid_positions)), top_new)
traj.save("13_solvent_1_new.pdb")


## THR->ALA solvent @ lambda = 0 WITHOUT flattened torsions and exceptions

In [34]:
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.alchemy import AbsoluteAlchemicalFactory, AlchemicalRegion, AlchemicalState
from openmmtools import states
from simtk import openmm, unit
from openmmtools.mcmc import LangevinSplittingDynamicsMove, GHMCMove
from openmmtools.multistate import ReplicaExchangeSampler, MultiStateReporter
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import numpy as np
import pickle

In [35]:
# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [36]:
# Generate htf for capped THR->ALA in solvent
pdb = app.PDBFile("../../input/thr_vacuum.pdb")

forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
system_generator = SystemGenerator(forcefields=forcefield_files,
                               barostat=barostat,
                               forcefield_kwargs={'removeCMMotion': False,
                                                    'ewaldErrorTolerance': 1e-4,
                                                    'constraints' : app.HBonds,
                                                    'hydrogenMass' : 4 * unit.amus},
                                periodic_forcefield_kwargs={'nonbondedMethod': app.PME},
                                small_molecule_forcefield='gaff-2.11',
                                nonperiodic_forcefield_kwargs=None, 
                                   molecules=None, 
                                   cache=None)
modeller = app.Modeller(pdb.topology, pdb.positions)
modeller.addSolvent(system_generator.forcefield, model='tip3p', padding=9*unit.angstroms, ionicStrength=0.15*unit.molar)
solvated_topology = modeller.getTopology()
solvated_positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
system = system_generator.create_system(solvated_topology)

htf = generate_dipeptide_top_pos_sys(solvated_topology, 
                                         new_res='ALA', 
                                         system=system, 
                                         positions=positions,
                                         system_generator=system_generator, 
                                         conduct_htf_prop=True,
                                         repartitioned=True,
                                         endstate=0,
                                         validate_endstate_energy=False)

INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{7: 9}, {8: 9}, {9: 9}, {7: 10}, {8: 10}, {9: 10}, {7: 11}, {8: 11}, {9: 11}, {7: 12}, {8: 12}, {9: 12}, {7: 13}, {8: 13}, {9: 13}]
INFO:proposal_generator:{7: 9}
INFO:proposal_generator:{8: 9}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{7: 10}
INFO:proposal_generator:{8: 10}
INFO:proposal_generator:{9: 10}
INFO:proposal_generator:{7: 11}
INFO:proposal_generator:{8: 11}
INFO:proposal_generator:{9: 11}
INFO:proposal_generator:{7: 12}
INFO:proposal_generator:{8: 12}
INFO:proposal_generator:{9: 12}
INFO:proposal_generator:{7: 13}
INFO:proposal_generator:{8: 13}
INFO:proposal_generator:{9: 13}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
INFO:proposal_generator:Finding best map using matching_criterion name
INFO:proposal_genera

making topology proposal


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 4
INFO:geometry:Atom index proposal order is [10, 13, 14, 15]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


generating geometry engine
making geometry proposal from THR to ALA


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 9 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 36 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 42 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1619 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (11, 6, 7, 10, [1, Quantity(value=0.97996

...

INFO:geometry:total reduced potential before atom placement: 9.117927746118994


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -72.55558356848493
INFO:geometry:final reduced energy -63.43765693685564
INFO:geometry:sum of energies: -63.43765582236594
INFO:geometry:magnitude of difference in the energies: 1.1144896916448488e-06
INFO:geometry:Final logp_proposal: -28861.495727571506


added energy components: [('CustomBondForce', 0.5174288400267416), ('CustomAngleForce', 6.782407624225441), ('CustomTorsionForce', 17.086151956175012), ('CustomBondForce', -96.94157198891212)]


INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.5648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.5648, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.5648), unit=nanometer)]
INFO:relative:Determined atom classes.
INFO:relative:Generating old system exceptions dict...
INFO:relative:Generating new system exceptions dict...
INFO:relative:Handling constraint

In [37]:
import pickle
import os
out_dir = '/data/chodera/zhangi/perses_benchmark/neq/10/14/'
pickle.dump(htf, open(os.path.join(out_dir, "14_solvent_0.pickle"), "wb" ))

# Render atom map
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping
atom_map_filename = os.path.join(out_dir, 'solvent_0_atom_map.png')
render_protein_residue_atom_mapping(htf._topology_proposal, atom_map_filename)


In [38]:
import mdtraj as md
import numpy as np
htfs_t = [htf]

top_old = md.Topology.from_openmm(htfs_t[0]._topology_proposal.old_topology)
top_new = md.Topology.from_openmm(htfs_t[0]._topology_proposal.new_topology)
traj = md.Trajectory(np.array(htfs_t[0].old_positions(htfs_t[0].hybrid_positions)), top_old)
traj.save("14_solvent_0_old.pdb")
traj = md.Trajectory(np.array(htfs_t[0].new_positions(htfs_t[0].hybrid_positions)), top_new)
traj.save("14_solvent_0_new.pdb")


## THR->ALA solvent @ lambda = 1 WITHOUT flattened torsions and exceptions

In [39]:
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.alchemy import AbsoluteAlchemicalFactory, AlchemicalRegion, AlchemicalState
from openmmtools import states
from simtk import openmm, unit
from openmmtools.mcmc import LangevinSplittingDynamicsMove, GHMCMove
from openmmtools.multistate import ReplicaExchangeSampler, MultiStateReporter
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import numpy as np
import pickle

In [40]:
# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [41]:
# Generate htf for capped THR->ALA in solvent
pdb = app.PDBFile("../../input/thr_vacuum.pdb")

forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
system_generator = SystemGenerator(forcefields=forcefield_files,
                               barostat=barostat,
                               forcefield_kwargs={'removeCMMotion': False,
                                                    'ewaldErrorTolerance': 1e-4,
                                                    'constraints' : app.HBonds,
                                                    'hydrogenMass' : 4 * unit.amus},
                                periodic_forcefield_kwargs={'nonbondedMethod': app.PME},
                                small_molecule_forcefield='gaff-2.11',
                                nonperiodic_forcefield_kwargs=None, 
                                   molecules=None, 
                                   cache=None)
modeller = app.Modeller(pdb.topology, pdb.positions)
modeller.addSolvent(system_generator.forcefield, model='tip3p', padding=9*unit.angstroms, ionicStrength=0.15*unit.molar)
solvated_topology = modeller.getTopology()
solvated_positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
system = system_generator.create_system(solvated_topology)

htf = generate_dipeptide_top_pos_sys(solvated_topology, 
                                         new_res='ALA', 
                                         system=system, 
                                         positions=positions,
                                         system_generator=system_generator, 
                                         conduct_htf_prop=True,
                                         repartitioned=True,
                                         endstate=1,
                                         validate_endstate_energy=False)

INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{7: 9}, {8: 9}, {9: 9}, {7: 10}, {8: 10}, {9: 10}, {7: 11}, {8: 11}, {9: 11}, {7: 12}, {8: 12}, {9: 12}, {7: 13}, {8: 13}, {9: 13}]
INFO:proposal_generator:{7: 9}
INFO:proposal_generator:{8: 9}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{7: 10}
INFO:proposal_generator:{8: 10}
INFO:proposal_generator:{9: 10}
INFO:proposal_generator:{7: 11}
INFO:proposal_generator:{8: 11}
INFO:proposal_generator:{9: 11}
INFO:proposal_generator:{7: 12}
INFO:proposal_generator:{8: 12}
INFO:proposal_generator:{9: 12}
INFO:proposal_generator:{7: 13}
INFO:proposal_generator:{8: 13}
INFO:proposal_generator:{9: 13}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
INFO:proposal_generator:Finding best map using matching_criterion name
INFO:proposal_genera

making topology proposal


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 4
INFO:geometry:Atom index proposal order is [10, 13, 14, 15]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


generating geometry engine
making geometry proposal from THR to ALA


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 9 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 36 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 42 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1619 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (11, 6, 7, 10, [1, Quantity(value=0.97996

...

INFO:geometry:total reduced potential before atom placement: 9.117927746118994


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -72.55558356848493
INFO:geometry:final reduced energy -63.43765693685564
INFO:geometry:sum of energies: -63.43765582236594
INFO:geometry:magnitude of difference in the energies: 1.1144896916448488e-06
INFO:geometry:Final logp_proposal: -28801.316251775825


added energy components: [('CustomBondForce', 0.5174288400267416), ('CustomAngleForce', 6.782407624225441), ('CustomTorsionForce', 17.086151956175012), ('CustomBondForce', -96.94157198891212)]


INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.5648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.5648, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.5648), unit=nanometer)]
INFO:relative:Determined atom classes.
INFO:relative:Generating old system exceptions dict...
INFO:relative:Generating new system exceptions dict...
INFO:relative:Handling constraint

In [42]:
import pickle
import os
out_dir = '/data/chodera/zhangi/perses_benchmark/neq/10/14/'
pickle.dump(htf, open(os.path.join(out_dir, "14_solvent_1.pickle"), "wb" ))

# Render atom map
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping
atom_map_filename = os.path.join(out_dir, 'solvent_1_atom_map.png')
render_protein_residue_atom_mapping(htf._topology_proposal, atom_map_filename)


In [43]:
import mdtraj as md
import numpy as np
htfs_t = [htf]

top_old = md.Topology.from_openmm(htfs_t[0]._topology_proposal.old_topology)
top_new = md.Topology.from_openmm(htfs_t[0]._topology_proposal.new_topology)
traj = md.Trajectory(np.array(htfs_t[0].old_positions(htfs_t[0].hybrid_positions)), top_old)
traj.save("14_solvent_1_old.pdb")
traj = md.Trajectory(np.array(htfs_t[0].new_positions(htfs_t[0].hybrid_positions)), top_new)
traj.save("14_solvent_1_new.pdb")


## 8-mer solvent @ lambda = 0 with flattened torsions and exceptions

In [1]:
from openmmtools.alchemy import AbsoluteAlchemicalFactory, AlchemicalRegion, AlchemicalState
from openmmtools import states
from simtk import openmm, unit
from openmmtools.mcmc import LangevinSplittingDynamicsMove, GHMCMove
from openmmtools.multistate import ReplicaExchangeSampler, MultiStateReporter
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import numpy as np
import pickle

In [2]:
# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [3]:
# Generate htf for 8-mer in solvent

from openmmtools.constants import kB
temperature = 300*unit.kelvin
# Compute kT and inverse temperature.
kT = kB * temperature
beta = 1.0 / kT

pdb = app.PDBFile("../../input/4zuh_peptide_capped.pdb")

forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
system_generator = SystemGenerator(forcefields=forcefield_files,
                               barostat=barostat,
                               forcefield_kwargs={'removeCMMotion': False,
                                                    'ewaldErrorTolerance':  0.00025,
                                                    'constraints' : app.HBonds,
                                                    'hydrogenMass' : 4 * unit.amus},
                                periodic_forcefield_kwargs={'nonbondedMethod': app.PME},
                                small_molecule_forcefield='gaff-2.11',
                                nonperiodic_forcefield_kwargs=None, 
                                   molecules=None, 
                                   cache=None)
modeller = app.Modeller(pdb.topology, pdb.positions)
# modeller.addHydrogens(forcefield=app.ForceField('amber14/protein.ff14SB.xml'))
modeller.addSolvent(system_generator.forcefield, model='tip3p', padding=9*unit.angstroms, ionicStrength=0.15*unit.molar)
solvated_topology = modeller.getTopology()
solvated_positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
system = system_generator.create_system(solvated_topology)

# Create the point mutation engine
from perses.rjmc.topology_proposal import PointMutationEngine
point_mutation_engine = PointMutationEngine(wildtype_topology=solvated_topology,
                                            system_generator=system_generator,
                                            chain_id='C', # Denote the chain id allowed to mutate (it's always a string variable)
                                            max_point_mutants=1,
                                            residues_allowed_to_mutate=['1'], # The residue ids allowed to mutate
                                            allowed_mutations=[('1', 'ALA')], # The residue ids allowed to mutate with the three-letter code allowed to change
                                            aggregate=True) # Always allow aggregation

# Create a top proposal
print(f"making topology proposal")
topology_proposal = point_mutation_engine.propose(current_system=system, current_topology=solvated_topology)


# Create a geometry engine
print(f"generating geometry engine")
from perses.rjmc.geometry import FFAllAngleGeometryEngine
geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                       use_sterics=False,
                                       n_bond_divisions=100,
                                       n_angle_divisions=180,
                                       n_torsion_divisions=360,
                                       verbose=True,
                                       storage=None,
                                       bond_softening_constant=1.0,
                                       angle_softening_constant=1.0,
                                       neglect_angles = False,
                                       use_14_nonbondeds = True)


# Make a geometry proposal forward
print(f"making geometry proposal from {list(solvated_topology.residues())[1].name} to ALA")
forward_new_positions, logp_proposal = geometry_engine.propose(topology_proposal, positions, beta, validate_energy_bookkeeping=True)
logp_reverse = geometry_engine.logp_reverse(topology_proposal, forward_new_positions, positions, beta, validate_energy_bookkeeping=True)

from perses.annihilation.relative import RepartitionedHybridTopologyFactory
factory = RepartitionedHybridTopologyFactory

htf = RepartitionedHybridTopologyFactory(topology_proposal=topology_proposal,
             current_positions=positions,
             new_positions=forward_new_positions,
             use_dispersion_correction=False,
             functions=None,
             softcore_alpha=None,
             bond_softening_constant=1.0,
             angle_softening_constant=1.0,
             soften_only_new=False,
             neglected_new_angle_terms=[],
             neglected_old_angle_terms=[],
             softcore_LJ_v2=True,
             softcore_electrostatics=True,
             softcore_LJ_v2_alpha=0.85,
             softcore_electrostatics_alpha=0.3,
             softcore_sigma_Q=1.0,
             interpolate_old_and_new_14s=True,
             omitted_terms=None,
             endstate=0,
             flatten_torsions=True)

        

/home/zhangi/miniconda3/envs/perses-sims-oct2020/lib/python3.7/site-packages/perses-0.8.2.dev0-py3.7.egg/perses/rjmc/coordinate_numba.py:14: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, A))
  n_2 = np.dot(a, a)
/home/zhangi/miniconda3/envs/perses-sims-oct2020/lib/python3.7/site-packages/perses-0.8.2.dev0-py3.7.egg/perses/rjmc/coordinate_numba.py:65: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 2d, A), array(float64, 1d, C))
  d_ang = np.dot(angle_rotation_matrix, d_r)
/home/zhangi/miniconda3/envs/perses-sims-oct2020/lib/python3.7/site-packages/perses-0.8.2.dev0-py3.7.egg/perses/rjmc/coordinate_numba.py:71: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 2d, A), array(float64, 1d, C))
  d_torsion = np.dot(torsion_rotation_matrix, d_ang)
/home/zhangi/miniconda3/envs/perses-sims-oct2020/lib/python3.7/site-packages/perses-0.8.2

making topology proposal


INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{7: 8}, {8: 8}, {9: 8}, {7: 9}, {8: 9}, {9: 9}, {7: 10}, {8: 10}, {9: 10}]
INFO:proposal_generator:{7: 8}
INFO:proposal_generator:{8: 8}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{7: 9}
INFO:proposal_generator:{8: 9}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{7: 10}
INFO:proposal_generator:{8: 10}
INFO:proposal_generator:{9: 10}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
INFO:proposal_generator:Finding best map using matching_criterion name
INFO:proposal_generator:{8: 8}
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 4
INFO:geometry:Atom index proposal

generating geometry engine
making geometry proposal from SER to ALA


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 60 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 217 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 387 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 6082 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (11, 6, 7, 10, [1, Quantity(value=0.97

...

INFO:geometry:total reduced potential before atom placement: 160.73588321674669


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 8.921633991620556
INFO:geometry:final reduced energy 169.65751611592427
INFO:geometry:sum of energies: 169.65751720836724
INFO:geometry:magnitude of difference in the energies: 1.092442975192398e-06
INFO:geometry:Final logp_proposal: -1706.3448278759925


added energy components: [('CustomBondForce', 0.008149351805885625), ('CustomAngleForce', 0.2838348278423751), ('CustomTorsionForce', 14.9612239690804), ('CustomBondForce', -6.331574157108108)]


INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=3.9005, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=3.9005, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=3.9005), unit=nanometer)]
INFO:relative:Determined atom classes.
INFO:

In [5]:
import pickle
import os
out_dir = '/data/chodera/zhangi/perses_benchmark/neq/10/15/'
pickle.dump(htf, open(os.path.join(out_dir, "15_solvent_0.pickle"), "wb" ))

# Render atom map
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping
atom_map_filename = os.path.join(out_dir, 'solvent_0_atom_map.png')
render_protein_residue_atom_mapping(htf._topology_proposal, atom_map_filename)


In [6]:
import mdtraj as md
import numpy as np
htfs_t = [htf]

top_old = md.Topology.from_openmm(htfs_t[0]._topology_proposal.old_topology)
top_new = md.Topology.from_openmm(htfs_t[0]._topology_proposal.new_topology)
traj = md.Trajectory(np.array(htfs_t[0].old_positions(htfs_t[0].hybrid_positions)), top_old)
traj.save("15_solvent_0_old.pdb")
traj = md.Trajectory(np.array(htfs_t[0].new_positions(htfs_t[0].hybrid_positions)), top_new)
traj.save("15_solvent_0_new.pdb")


## 8-mer solvent @ lambda = 1 with flattened torsions and exceptions

In [7]:
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.alchemy import AbsoluteAlchemicalFactory, AlchemicalRegion, AlchemicalState
from openmmtools import states
from simtk import openmm, unit
from openmmtools.mcmc import LangevinSplittingDynamicsMove, GHMCMove
from openmmtools.multistate import ReplicaExchangeSampler, MultiStateReporter
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import numpy as np
import pickle

INFO:rdkit:Enabling RDKit 2020.09.1 jupyter extensions


In [8]:
# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [9]:
# Generate htf for 8-mer in solvent

from openmmtools.constants import kB
temperature = 300*unit.kelvin
# Compute kT and inverse temperature.
kT = kB * temperature
beta = 1.0 / kT

pdb = app.PDBFile("../../input/4zuh_peptide_capped.pdb")

forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
system_generator = SystemGenerator(forcefields=forcefield_files,
                               barostat=barostat,
                               forcefield_kwargs={'removeCMMotion': False,
                                                    'ewaldErrorTolerance':  0.00025,
                                                    'constraints' : app.HBonds,
                                                    'hydrogenMass' : 4 * unit.amus},
                                periodic_forcefield_kwargs={'nonbondedMethod': app.PME},
                                small_molecule_forcefield='gaff-2.11',
                                nonperiodic_forcefield_kwargs=None, 
                                   molecules=None, 
                                   cache=None)
modeller = app.Modeller(pdb.topology, pdb.positions)
# modeller.addHydrogens(forcefield=app.ForceField('amber14/protein.ff14SB.xml'))
modeller.addSolvent(system_generator.forcefield, model='tip3p', padding=9*unit.angstroms, ionicStrength=0.15*unit.molar)
solvated_topology = modeller.getTopology()
solvated_positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
system = system_generator.create_system(solvated_topology)

# Create the point mutation engine
from perses.rjmc.topology_proposal import PointMutationEngine
point_mutation_engine = PointMutationEngine(wildtype_topology=solvated_topology,
                                            system_generator=system_generator,
                                            chain_id='C', # Denote the chain id allowed to mutate (it's always a string variable)
                                            max_point_mutants=1,
                                            residues_allowed_to_mutate=['1'], # The residue ids allowed to mutate
                                            allowed_mutations=[('1', 'ALA')], # The residue ids allowed to mutate with the three-letter code allowed to change
                                            aggregate=True) # Always allow aggregation

# Create a top proposal
print(f"making topology proposal")
topology_proposal = point_mutation_engine.propose(current_system=system, current_topology=solvated_topology)


# Create a geometry engine
print(f"generating geometry engine")
from perses.rjmc.geometry import FFAllAngleGeometryEngine
geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                       use_sterics=False,
                                       n_bond_divisions=100,
                                       n_angle_divisions=180,
                                       n_torsion_divisions=360,
                                       verbose=True,
                                       storage=None,
                                       bond_softening_constant=1.0,
                                       angle_softening_constant=1.0,
                                       neglect_angles = False,
                                       use_14_nonbondeds = True)


# Make a geometry proposal forward
print(f"making geometry proposal from {list(solvated_topology.residues())[1].name} to ALA")
forward_new_positions, logp_proposal = geometry_engine.propose(topology_proposal, positions, beta, validate_energy_bookkeeping=True)
logp_reverse = geometry_engine.logp_reverse(topology_proposal, forward_new_positions, positions, beta, validate_energy_bookkeeping=True)

from perses.annihilation.relative import RepartitionedHybridTopologyFactory
factory = RepartitionedHybridTopologyFactory

htf = RepartitionedHybridTopologyFactory(topology_proposal=topology_proposal,
             current_positions=positions,
             new_positions=forward_new_positions,
             use_dispersion_correction=False,
             functions=None,
             softcore_alpha=None,
             bond_softening_constant=1.0,
             angle_softening_constant=1.0,
             soften_only_new=False,
             neglected_new_angle_terms=[],
             neglected_old_angle_terms=[],
             softcore_LJ_v2=True,
             softcore_electrostatics=True,
             softcore_LJ_v2_alpha=0.85,
             softcore_electrostatics_alpha=0.3,
             softcore_sigma_Q=1.0,
             interpolate_old_and_new_14s=True,
             omitted_terms=None,
             endstate=1,
             flatten_torsions=True)

        

INFO:proposal_generator:	Conducting polymer point mutation proposal...


making topology proposal


INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{7: 8}, {8: 8}, {9: 8}, {7: 9}, {8: 9}, {9: 9}, {7: 10}, {8: 10}, {9: 10}]
INFO:proposal_generator:{7: 8}
INFO:proposal_generator:{8: 8}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{7: 9}
INFO:proposal_generator:{8: 9}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{7: 10}
INFO:proposal_generator:{8: 10}
INFO:proposal_generator:{9: 10}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
INFO:proposal_generator:Finding best map using matching_criterion name
INFO:proposal_generator:{8: 8}
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 4
INFO:geometry:Atom index proposal

generating geometry engine
making geometry proposal from SER to ALA


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 60 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 217 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 387 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 6082 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (11, 6, 7, 10, [1, Quantity(value=0.97

...

INFO:geometry:total reduced potential before atom placement: 160.73588321674669


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 8.921633991620556
INFO:geometry:final reduced energy 169.65751611592427
INFO:geometry:sum of energies: 169.65751720836724
INFO:geometry:magnitude of difference in the energies: 1.092442975192398e-06
INFO:geometry:Final logp_proposal: -1701.7198137521677


added energy components: [('CustomBondForce', 0.008149351805885625), ('CustomAngleForce', 0.2838348278423751), ('CustomTorsionForce', 14.9612239690804), ('CustomBondForce', -6.331574157108108)]


INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=3.9005, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=3.9005, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=3.9005), unit=nanometer)]
INFO:relative:Determined atom classes.
INFO:

In [10]:
import pickle
import os
out_dir = '/data/chodera/zhangi/perses_benchmark/neq/10/15/'
pickle.dump(htf, open(os.path.join(out_dir, "15_solvent_1.pickle"), "wb" ))

# Render atom map
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping
atom_map_filename = os.path.join(out_dir, 'solvent_1_atom_map.png')
render_protein_residue_atom_mapping(htf._topology_proposal, atom_map_filename)


In [11]:
import mdtraj as md
import numpy as np
htfs_t = [htf]

top_old = md.Topology.from_openmm(htfs_t[0]._topology_proposal.old_topology)
top_new = md.Topology.from_openmm(htfs_t[0]._topology_proposal.new_topology)
traj = md.Trajectory(np.array(htfs_t[0].old_positions(htfs_t[0].hybrid_positions)), top_old)
traj.save("15_solvent_1_old.pdb")
traj = md.Trajectory(np.array(htfs_t[0].new_positions(htfs_t[0].hybrid_positions)), top_new)
traj.save("15_solvent_1_new.pdb")


In [2]:
import pickle
with open("/data/chodera/zhangi/perses_benchmark/neq/10/15/15_solvent_1.pickle", "rb") as f:
    htf = pickle.load(f)

In [4]:
for atom in htf._topology_proposal.old_topology.atoms():
    print(atom)

<Atom 0 (CH3) of chain 0 residue 0 (ACE)>
<Atom 1 (C) of chain 0 residue 0 (ACE)>
<Atom 2 (O) of chain 0 residue 0 (ACE)>
<Atom 3 (H1) of chain 0 residue 0 (ACE)>
<Atom 4 (H2) of chain 0 residue 0 (ACE)>
<Atom 5 (H3) of chain 0 residue 0 (ACE)>
<Atom 6 (N) of chain 0 residue 1 (SER)>
<Atom 7 (CA) of chain 0 residue 1 (SER)>
<Atom 8 (C) of chain 0 residue 1 (SER)>
<Atom 9 (O) of chain 0 residue 1 (SER)>
<Atom 10 (CB) of chain 0 residue 1 (SER)>
<Atom 11 (OG) of chain 0 residue 1 (SER)>
<Atom 12 (H) of chain 0 residue 1 (SER)>
<Atom 13 (HA) of chain 0 residue 1 (SER)>
<Atom 14 (HB3) of chain 0 residue 1 (SER)>
<Atom 15 (HB2) of chain 0 residue 1 (SER)>
<Atom 16 (HG) of chain 0 residue 1 (SER)>
<Atom 17 (N) of chain 0 residue 2 (ALA)>
<Atom 18 (CA) of chain 0 residue 2 (ALA)>
<Atom 19 (C) of chain 0 residue 2 (ALA)>
<Atom 20 (O) of chain 0 residue 2 (ALA)>
<Atom 21 (CB) of chain 0 residue 2 (ALA)>
<Atom 22 (H) of chain 0 residue 2 (ALA)>
<Atom 23 (HA) of chain 0 residue 2 (ALA)>
<Atom 24 

<Atom 1964 (H2) of chain 1 residue 623 (HOH)>
<Atom 1965 (O) of chain 1 residue 624 (HOH)>
<Atom 1966 (H1) of chain 1 residue 624 (HOH)>
<Atom 1967 (H2) of chain 1 residue 624 (HOH)>
<Atom 1968 (O) of chain 1 residue 625 (HOH)>
<Atom 1969 (H1) of chain 1 residue 625 (HOH)>
<Atom 1970 (H2) of chain 1 residue 625 (HOH)>
<Atom 1971 (O) of chain 1 residue 626 (HOH)>
<Atom 1972 (H1) of chain 1 residue 626 (HOH)>
<Atom 1973 (H2) of chain 1 residue 626 (HOH)>
<Atom 1974 (O) of chain 1 residue 627 (HOH)>
<Atom 1975 (H1) of chain 1 residue 627 (HOH)>
<Atom 1976 (H2) of chain 1 residue 627 (HOH)>
<Atom 1977 (O) of chain 1 residue 628 (HOH)>
<Atom 1978 (H1) of chain 1 residue 628 (HOH)>
<Atom 1979 (H2) of chain 1 residue 628 (HOH)>
<Atom 1980 (O) of chain 1 residue 629 (HOH)>
<Atom 1981 (H1) of chain 1 residue 629 (HOH)>
<Atom 1982 (H2) of chain 1 residue 629 (HOH)>
<Atom 1983 (O) of chain 1 residue 630 (HOH)>
<Atom 1984 (H1) of chain 1 residue 630 (HOH)>
<Atom 1985 (H2) of chain 1 residue 630 (H

<Atom 3463 (H1) of chain 1 residue 1123 (HOH)>
<Atom 3464 (H2) of chain 1 residue 1123 (HOH)>
<Atom 3465 (O) of chain 1 residue 1124 (HOH)>
<Atom 3466 (H1) of chain 1 residue 1124 (HOH)>
<Atom 3467 (H2) of chain 1 residue 1124 (HOH)>
<Atom 3468 (O) of chain 1 residue 1125 (HOH)>
<Atom 3469 (H1) of chain 1 residue 1125 (HOH)>
<Atom 3470 (H2) of chain 1 residue 1125 (HOH)>
<Atom 3471 (O) of chain 1 residue 1126 (HOH)>
<Atom 3472 (H1) of chain 1 residue 1126 (HOH)>
<Atom 3473 (H2) of chain 1 residue 1126 (HOH)>
<Atom 3474 (O) of chain 1 residue 1127 (HOH)>
<Atom 3475 (H1) of chain 1 residue 1127 (HOH)>
<Atom 3476 (H2) of chain 1 residue 1127 (HOH)>
<Atom 3477 (O) of chain 1 residue 1128 (HOH)>
<Atom 3478 (H1) of chain 1 residue 1128 (HOH)>
<Atom 3479 (H2) of chain 1 residue 1128 (HOH)>
<Atom 3480 (O) of chain 1 residue 1129 (HOH)>
<Atom 3481 (H1) of chain 1 residue 1129 (HOH)>
<Atom 3482 (H2) of chain 1 residue 1129 (HOH)>
<Atom 3483 (O) of chain 1 residue 1130 (HOH)>
<Atom 3484 (H1) of c

<Atom 4963 (H1) of chain 1 residue 1623 (HOH)>
<Atom 4964 (H2) of chain 1 residue 1623 (HOH)>
<Atom 4965 (O) of chain 1 residue 1624 (HOH)>
<Atom 4966 (H1) of chain 1 residue 1624 (HOH)>
<Atom 4967 (H2) of chain 1 residue 1624 (HOH)>
<Atom 4968 (O) of chain 1 residue 1625 (HOH)>
<Atom 4969 (H1) of chain 1 residue 1625 (HOH)>
<Atom 4970 (H2) of chain 1 residue 1625 (HOH)>
<Atom 4971 (O) of chain 1 residue 1626 (HOH)>
<Atom 4972 (H1) of chain 1 residue 1626 (HOH)>
<Atom 4973 (H2) of chain 1 residue 1626 (HOH)>
<Atom 4974 (O) of chain 1 residue 1627 (HOH)>
<Atom 4975 (H1) of chain 1 residue 1627 (HOH)>
<Atom 4976 (H2) of chain 1 residue 1627 (HOH)>
<Atom 4977 (O) of chain 1 residue 1628 (HOH)>
<Atom 4978 (H1) of chain 1 residue 1628 (HOH)>
<Atom 4979 (H2) of chain 1 residue 1628 (HOH)>
<Atom 4980 (O) of chain 1 residue 1629 (HOH)>
<Atom 4981 (H1) of chain 1 residue 1629 (HOH)>
<Atom 4982 (H2) of chain 1 residue 1629 (HOH)>
<Atom 4983 (O) of chain 1 residue 1630 (HOH)>
<Atom 4984 (H1) of c

In [5]:
for atom in htf._topology_proposal.new_topology.atoms():
    print(atom)

<Atom 0 (CH3) of chain 0 residue 0 (ACE)>
<Atom 1 (C) of chain 0 residue 0 (ACE)>
<Atom 2 (O) of chain 0 residue 0 (ACE)>
<Atom 3 (H1) of chain 0 residue 0 (ACE)>
<Atom 4 (H2) of chain 0 residue 0 (ACE)>
<Atom 5 (H3) of chain 0 residue 0 (ACE)>
<Atom 6 (N) of chain 0 residue 1 (ALA)>
<Atom 7 (CA) of chain 0 residue 1 (ALA)>
<Atom 8 (C) of chain 0 residue 1 (ALA)>
<Atom 9 (O) of chain 0 residue 1 (ALA)>
<Atom 10 (CB) of chain 0 residue 1 (ALA)>
<Atom 11 (H) of chain 0 residue 1 (ALA)>
<Atom 12 (HA) of chain 0 residue 1 (ALA)>
<Atom 13 (HB3) of chain 0 residue 1 (ALA)>
<Atom 14 (HB2) of chain 0 residue 1 (ALA)>
<Atom 15 (HB1) of chain 0 residue 1 (ALA)>
<Atom 16 (N) of chain 0 residue 2 (ALA)>
<Atom 17 (CA) of chain 0 residue 2 (ALA)>
<Atom 18 (C) of chain 0 residue 2 (ALA)>
<Atom 19 (O) of chain 0 residue 2 (ALA)>
<Atom 20 (CB) of chain 0 residue 2 (ALA)>
<Atom 21 (H) of chain 0 residue 2 (ALA)>
<Atom 22 (HA) of chain 0 residue 2 (ALA)>
<Atom 23 (HB1) of chain 0 residue 2 (ALA)>
<Atom 2

<Atom 1881 (H1) of chain 1 residue 596 (HOH)>
<Atom 1882 (H2) of chain 1 residue 596 (HOH)>
<Atom 1883 (O) of chain 1 residue 597 (HOH)>
<Atom 1884 (H1) of chain 1 residue 597 (HOH)>
<Atom 1885 (H2) of chain 1 residue 597 (HOH)>
<Atom 1886 (O) of chain 1 residue 598 (HOH)>
<Atom 1887 (H1) of chain 1 residue 598 (HOH)>
<Atom 1888 (H2) of chain 1 residue 598 (HOH)>
<Atom 1889 (O) of chain 1 residue 599 (HOH)>
<Atom 1890 (H1) of chain 1 residue 599 (HOH)>
<Atom 1891 (H2) of chain 1 residue 599 (HOH)>
<Atom 1892 (O) of chain 1 residue 600 (HOH)>
<Atom 1893 (H1) of chain 1 residue 600 (HOH)>
<Atom 1894 (H2) of chain 1 residue 600 (HOH)>
<Atom 1895 (O) of chain 1 residue 601 (HOH)>
<Atom 1896 (H1) of chain 1 residue 601 (HOH)>
<Atom 1897 (H2) of chain 1 residue 601 (HOH)>
<Atom 1898 (O) of chain 1 residue 602 (HOH)>
<Atom 1899 (H1) of chain 1 residue 602 (HOH)>
<Atom 1900 (H2) of chain 1 residue 602 (HOH)>
<Atom 1901 (O) of chain 1 residue 603 (HOH)>
<Atom 1902 (H1) of chain 1 residue 603 (H

<Atom 3381 (H1) of chain 1 residue 1096 (HOH)>
<Atom 3382 (H2) of chain 1 residue 1096 (HOH)>
<Atom 3383 (O) of chain 1 residue 1097 (HOH)>
<Atom 3384 (H1) of chain 1 residue 1097 (HOH)>
<Atom 3385 (H2) of chain 1 residue 1097 (HOH)>
<Atom 3386 (O) of chain 1 residue 1098 (HOH)>
<Atom 3387 (H1) of chain 1 residue 1098 (HOH)>
<Atom 3388 (H2) of chain 1 residue 1098 (HOH)>
<Atom 3389 (O) of chain 1 residue 1099 (HOH)>
<Atom 3390 (H1) of chain 1 residue 1099 (HOH)>
<Atom 3391 (H2) of chain 1 residue 1099 (HOH)>
<Atom 3392 (O) of chain 1 residue 1100 (HOH)>
<Atom 3393 (H1) of chain 1 residue 1100 (HOH)>
<Atom 3394 (H2) of chain 1 residue 1100 (HOH)>
<Atom 3395 (O) of chain 1 residue 1101 (HOH)>
<Atom 3396 (H1) of chain 1 residue 1101 (HOH)>
<Atom 3397 (H2) of chain 1 residue 1101 (HOH)>
<Atom 3398 (O) of chain 1 residue 1102 (HOH)>
<Atom 3399 (H1) of chain 1 residue 1102 (HOH)>
<Atom 3400 (H2) of chain 1 residue 1102 (HOH)>
<Atom 3401 (O) of chain 1 residue 1103 (HOH)>
<Atom 3402 (H1) of c

<Atom 4880 (O) of chain 1 residue 1596 (HOH)>
<Atom 4881 (H1) of chain 1 residue 1596 (HOH)>
<Atom 4882 (H2) of chain 1 residue 1596 (HOH)>
<Atom 4883 (O) of chain 1 residue 1597 (HOH)>
<Atom 4884 (H1) of chain 1 residue 1597 (HOH)>
<Atom 4885 (H2) of chain 1 residue 1597 (HOH)>
<Atom 4886 (O) of chain 1 residue 1598 (HOH)>
<Atom 4887 (H1) of chain 1 residue 1598 (HOH)>
<Atom 4888 (H2) of chain 1 residue 1598 (HOH)>
<Atom 4889 (O) of chain 1 residue 1599 (HOH)>
<Atom 4890 (H1) of chain 1 residue 1599 (HOH)>
<Atom 4891 (H2) of chain 1 residue 1599 (HOH)>
<Atom 4892 (O) of chain 1 residue 1600 (HOH)>
<Atom 4893 (H1) of chain 1 residue 1600 (HOH)>
<Atom 4894 (H2) of chain 1 residue 1600 (HOH)>
<Atom 4895 (O) of chain 1 residue 1601 (HOH)>
<Atom 4896 (H1) of chain 1 residue 1601 (HOH)>
<Atom 4897 (H2) of chain 1 residue 1601 (HOH)>
<Atom 4898 (O) of chain 1 residue 1602 (HOH)>
<Atom 4899 (H1) of chain 1 residue 1602 (HOH)>
<Atom 4900 (H2) of chain 1 residue 1602 (HOH)>
<Atom 4901 (O) of ch

## THR->ALA vacuum @ lambda = 0 with flattened torsions and exceptions

In [1]:
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.alchemy import AbsoluteAlchemicalFactory, AlchemicalRegion, AlchemicalState
from openmmtools import states
from simtk import openmm, unit
from openmmtools.mcmc import LangevinSplittingDynamicsMove, GHMCMove
from openmmtools.multistate import ReplicaExchangeSampler, MultiStateReporter
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import numpy as np
import pickle

INFO:rdkit:Enabling RDKit 2020.09.1 jupyter extensions


In [2]:
# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [3]:
# Generate htf for capped THR->ALA in solvent
pdb = app.PDBFile("../../input/thr_vacuum.pdb")

forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = None
system_generator = SystemGenerator(forcefields=forcefield_files,
                               barostat=barostat,
                               forcefield_kwargs={'removeCMMotion': False,
                                                    'ewaldErrorTolerance': 1e-4,
                                                    'constraints' : app.HBonds,
                                                    'hydrogenMass' : 4 * unit.amus},
                                periodic_forcefield_kwargs=None,
                                small_molecule_forcefield='gaff-2.11',
                                nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff}, 
                                   molecules=None, 
                                   cache=None)

# Canonicalize the solvated positions: turn tuples into np.array
positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in pdb.positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
system = system_generator.create_system(pdb.topology)

htf = generate_dipeptide_top_pos_sys(pdb.topology, 
                                         new_res='ALA', 
                                         system=system, 
                                         positions=positions,
                                         system_generator=system_generator, 
                                         conduct_htf_prop=True,
                                         repartitioned=True,
                                         endstate=0,
                                         flatten_torsions=True,
                                         flatten_exceptions=True,
                                         validate_endstate_energy=False)

INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{7: 9}, {8: 9}, {9: 9}, {7: 10}, {8: 10}, {9: 10}, {7: 11}, {8: 11}, {9: 11}, {7: 12}, {8: 12}, {9: 12}, {7: 13}, {8: 13}, {9: 13}]
INFO:proposal_generator:{7: 9}
INFO:proposal_generator:{8: 9}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{7: 10}
INFO:proposal_generator:{8: 10}
INFO:proposal_generator:{9: 10}
INFO:proposal_generator:{7: 11}
INFO:proposal_generator:{8: 11}
INFO:proposal_generator:{9: 11}
INFO:proposal_generator:{7: 12}
INFO:proposal_generator:{8: 12}
INFO:proposal_generator:{9: 12}
INFO:proposal_generator:{7: 13}
INFO:proposal_generator:{8: 13}
INFO:proposal_generator:{9: 13}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
INFO:proposal_generator:Finding best map using matching_criterion name
INFO:proposal_genera

making topology proposal
generating geometry engine
making geometry proposal from THR to ALA


INFO:geometry:creating platform, integrators, and contexts; setting growth parameter


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


...

INFO:geometry:total reduced potential before atom placement: 9.117927746118994


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -72.55558356848493
INFO:geometry:final reduced energy -63.437656936855454
INFO:geometry:sum of energies: -63.43765582236594
INFO:geometry:magnitude of difference in the energies: 1.1144895211145922e-06
INFO:geometry:Final logp_proposal: -28363.034341230006


added energy components: [('CustomBondForce', 0.5174288400267416), ('CustomAngleForce', 6.782407624225441), ('CustomTorsionForce', 17.086151956175012), ('CustomBondForce', -96.94157198891212)]


INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:relative:Determined atom classes.
INFO:relative:Generating old system exceptions dict...


In [4]:
import pickle
import os
out_dir = '/data/chodera/zhangi/perses_benchmark/neq/10/16/'
pickle.dump(htf, open(os.path.join(out_dir, "16_vacuum_0.pickle"), "wb" ))

# Render atom map
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping
atom_map_filename = os.path.join(out_dir, 'vacuum_0_atom_map.png')
render_protein_residue_atom_mapping(htf._topology_proposal, atom_map_filename)


In [5]:
import mdtraj as md
import numpy as np
htfs_t = [htf]

top_old = md.Topology.from_openmm(htfs_t[0]._topology_proposal.old_topology)
top_new = md.Topology.from_openmm(htfs_t[0]._topology_proposal.new_topology)
traj = md.Trajectory(np.array(htfs_t[0].old_positions(htfs_t[0].hybrid_positions)), top_old)
traj.save("16_solvent_0_old.pdb")
traj = md.Trajectory(np.array(htfs_t[0].new_positions(htfs_t[0].hybrid_positions)), top_new)
traj.save("16_solvent_0_new.pdb")


## THR->ALA vacuum @ lambda = 1 with flattened torsions and exceptions

In [6]:
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.alchemy import AbsoluteAlchemicalFactory, AlchemicalRegion, AlchemicalState
from openmmtools import states
from simtk import openmm, unit
from openmmtools.mcmc import LangevinSplittingDynamicsMove, GHMCMove
from openmmtools.multistate import ReplicaExchangeSampler, MultiStateReporter
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import numpy as np
import pickle

In [7]:
# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

In [8]:
# Generate htf for capped THR->ALA in solvent
pdb = app.PDBFile("../../input/thr_vacuum.pdb")

forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = None
system_generator = SystemGenerator(forcefields=forcefield_files,
                               barostat=barostat,
                               forcefield_kwargs={'removeCMMotion': False,
                                                    'ewaldErrorTolerance': 1e-4,
                                                    'constraints' : app.HBonds,
                                                    'hydrogenMass' : 4 * unit.amus},
                                periodic_forcefield_kwargs=None,
                                small_molecule_forcefield='gaff-2.11',
                                nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff}, 
                                   molecules=None, 
                                   cache=None)

# Canonicalize the solvated positions: turn tuples into np.array
positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in pdb.positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
system = system_generator.create_system(pdb.topology)

htf = generate_dipeptide_top_pos_sys(pdb.topology, 
                                         new_res='ALA', 
                                         system=system, 
                                         positions=positions,
                                         system_generator=system_generator, 
                                         conduct_htf_prop=True,
                                         repartitioned=True,
                                         endstate=1,
                                         flatten_torsions=True,
                                         flatten_exceptions=True,
                                         validate_endstate_energy=False)

INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{7: 9}, {8: 9}, {9: 9}, {7: 10}, {8: 10}, {9: 10}, {7: 11}, {8: 11}, {9: 11}, {7: 12}, {8: 12}, {9: 12}, {7: 13}, {8: 13}, {9: 13}]
INFO:proposal_generator:{7: 9}
INFO:proposal_generator:{8: 9}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{7: 10}
INFO:proposal_generator:{8: 10}
INFO:proposal_generator:{9: 10}
INFO:proposal_generator:{7: 11}
INFO:proposal_generator:{8: 11}
INFO:proposal_generator:{9: 11}
INFO:proposal_generator:{7: 12}
INFO:proposal_generator:{8: 12}
INFO:proposal_generator:{9: 12}
INFO:proposal_generator:{7: 13}
INFO:proposal_generator:{8: 13}
INFO:proposal_generator:{9: 13}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
INFO:proposal_generator:Finding best map using matching_criterion name
INFO:proposal_genera

making topology proposal
generating geometry engine
making geometry proposal from THR to ALA
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 4 new atoms
INFO:geometry:	reduced angle potential = 0.29659320346299717.
INFO:geometry:	reduced angle potential = 0.8393937369009811.
INFO:geometry:	reduced angle potential = 0.16760267301059895.
INFO:geometry:	reduced angle potential = 0.3347524272719817.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 9 bond forces in the no-nonbonded final system
INFO:geometry:	there are 36 angle forces in the no-nonbonded final system
INFO:geometry:	there are 42 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.


...

INFO:geometry:total reduced potential before atom placement: 9.117927746118994


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -72.55558356848493
INFO:geometry:final reduced energy -63.437656936855454
INFO:geometry:sum of energies: -63.43765582236594
INFO:geometry:magnitude of difference in the energies: 1.1144895211145922e-06
INFO:geometry:Final logp_proposal: -28868.555321091695


added energy components: [('CustomBondForce', 0.5174288400267416), ('CustomAngleForce', 6.782407624225441), ('CustomTorsionForce', 17.086151956175012), ('CustomBondForce', -96.94157198891212)]


INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:relative:Determined atom classes.
INFO:relative:Generating old system exceptions dict...


In [9]:
import pickle
import os
out_dir = '/data/chodera/zhangi/perses_benchmark/neq/10/16/'
pickle.dump(htf, open(os.path.join(out_dir, "16_vacuum_1.pickle"), "wb" ))

# Render atom map
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping
atom_map_filename = os.path.join(out_dir, 'vacuum_1_atom_map.png')
render_protein_residue_atom_mapping(htf._topology_proposal, atom_map_filename)


In [10]:
import mdtraj as md
import numpy as np
htfs_t = [htf]

top_old = md.Topology.from_openmm(htfs_t[0]._topology_proposal.old_topology)
top_new = md.Topology.from_openmm(htfs_t[0]._topology_proposal.new_topology)
traj = md.Trajectory(np.array(htfs_t[0].old_positions(htfs_t[0].hybrid_positions)), top_old)
traj.save("16_solvent_1_old.pdb")
traj = md.Trajectory(np.array(htfs_t[0].new_positions(htfs_t[0].hybrid_positions)), top_new)
traj.save("16_solvent_1_new.pdb")


In [2]:
with open("/data/chodera/zhangi/perses_benchmark/neq/10/16/16_vacuum_0.pickle", "rb") as f:
    htf = pickle.load(f)

In [14]:
htf._atom_classes['unique_old_atoms']

{10, 11, 12, 15, 16, 17, 18, 19}

In [16]:
htf._atom_classes['unique_new_atoms']

{26, 27, 28, 29}

In [15]:
for atom in htf.hybrid_topology.atoms:
    print(atom, atom.index)

ACE1-C 0
ACE1-O 1
ACE1-CH3 2
ACE1-H1 3
ACE1-H2 4
ACE1-H3 5
THR2-N 6
THR2-CA 7
THR2-C 8
THR2-O 9
THR2-CB 10
THR2-CG2 11
THR2-OG1 12
THR2-H 13
THR2-HA 14
THR2-HB 15
THR2-HG1 16
THR2-HG21 17
THR2-HG22 18
THR2-HG23 19
THR2-CB 26
THR2-HB1 27
THR2-HB2 28
THR2-HB3 29
NME3-N 20
NME3-C 21
NME3-H 22
NME3-H1 23
NME3-H2 24
NME3-H3 25


In [ ]:
# lambda = 0 
# whole residue + neighboring N/C: 6, 7, 8, 9, 13, 14, 20, 21
# all protein atoms: 0-29

# lambda = 0
# whole residue + neighboring N/C: 6, 7, 8, 9, 13, 14, 20, 21
# all protein atoms: 0-29

In [3]:
for atom in htf._topology_proposal.old_topology.atoms():
    print(atom, atom.index)

<Atom 0 (C) of chain 0 residue 0 (ACE)> 0
<Atom 1 (O) of chain 0 residue 0 (ACE)> 1
<Atom 2 (CH3) of chain 0 residue 0 (ACE)> 2
<Atom 3 (H1) of chain 0 residue 0 (ACE)> 3
<Atom 4 (H2) of chain 0 residue 0 (ACE)> 4
<Atom 5 (H3) of chain 0 residue 0 (ACE)> 5
<Atom 6 (N) of chain 0 residue 1 (THR)> 6
<Atom 7 (CA) of chain 0 residue 1 (THR)> 7
<Atom 8 (C) of chain 0 residue 1 (THR)> 8
<Atom 9 (O) of chain 0 residue 1 (THR)> 9
<Atom 10 (CB) of chain 0 residue 1 (THR)> 10
<Atom 11 (CG2) of chain 0 residue 1 (THR)> 11
<Atom 12 (OG1) of chain 0 residue 1 (THR)> 12
<Atom 13 (H) of chain 0 residue 1 (THR)> 13
<Atom 14 (HA) of chain 0 residue 1 (THR)> 14
<Atom 15 (HB) of chain 0 residue 1 (THR)> 15
<Atom 16 (HG1) of chain 0 residue 1 (THR)> 16
<Atom 17 (HG21) of chain 0 residue 1 (THR)> 17
<Atom 18 (HG22) of chain 0 residue 1 (THR)> 18
<Atom 19 (HG23) of chain 0 residue 1 (THR)> 19
<Atom 20 (N) of chain 0 residue 2 (NME)> 20
<Atom 21 (C) of chain 0 residue 2 (NME)> 21
<Atom 22 (H) of chain 0 resi

In [4]:
for atom in htf._topology_proposal.new_topology.atoms():
    print(atom, atom.index)

<Atom 0 (C) of chain 0 residue 0 (ACE)> 0
<Atom 1 (O) of chain 0 residue 0 (ACE)> 1
<Atom 2 (CH3) of chain 0 residue 0 (ACE)> 2
<Atom 3 (H1) of chain 0 residue 0 (ACE)> 3
<Atom 4 (H2) of chain 0 residue 0 (ACE)> 4
<Atom 5 (H3) of chain 0 residue 0 (ACE)> 5
<Atom 6 (N) of chain 0 residue 1 (ALA)> 6
<Atom 7 (CA) of chain 0 residue 1 (ALA)> 7
<Atom 8 (C) of chain 0 residue 1 (ALA)> 8
<Atom 9 (O) of chain 0 residue 1 (ALA)> 9
<Atom 10 (CB) of chain 0 residue 1 (ALA)> 10
<Atom 11 (H) of chain 0 residue 1 (ALA)> 11
<Atom 12 (HA) of chain 0 residue 1 (ALA)> 12
<Atom 13 (HB1) of chain 0 residue 1 (ALA)> 13
<Atom 14 (HB2) of chain 0 residue 1 (ALA)> 14
<Atom 15 (HB3) of chain 0 residue 1 (ALA)> 15
<Atom 16 (N) of chain 0 residue 2 (NME)> 16
<Atom 17 (C) of chain 0 residue 2 (NME)> 17
<Atom 18 (H) of chain 0 residue 2 (NME)> 18
<Atom 19 (H1) of chain 0 residue 2 (NME)> 19
<Atom 20 (H2) of chain 0 residue 2 (NME)> 20
<Atom 21 (H3) of chain 0 residue 2 (NME)> 21
